In [ ]:
import numpy as np
import pandas as pd
import os
import json
import tqdm
import toml

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
base_path = "/content/drive/MyDrive/Colab Notebooks/20-30 Oct 2024- Test Data Algo1-2-3_mlp"

In [ ]:
# Combine the C_train, C_test, C_val generated from 3-Default LLM Prompt for each row with then N_train, N_test, N_val with original data because now everything is numeric
# Also Create info.json files
# Here basically we are dealing with folders such as adult_algo_3, bank_marketing_algo_3

algo_list = ["3_Embedding_Prompt_for_each_row"]
data_list = ["adult", "analcatdata", "bank_marketing", "credit-approval", "credit-g", "cylinder-bands", "dresses-sales", "eucalyptus", "kr-vs-kp", "nursery", "titanic", "sick"]
for algo in algo_list:
  for data in tqdm.tqdm(data_list):
    dir_name = f"{data}_algo_{algo.split('_')[0]}"
    path = f"{base_path}/Final Data/{dir_name}"
    os.makedirs(path, exist_ok=True)

    y_train = np.load(f"{base_path}/Benchmark Data/{data}/y_train.npy", allow_pickle=True)
    y_train_df = pd.DataFrame(y_train)

    y_test = np.load(f"{base_path}/Benchmark Data/{data}/y_test.npy", allow_pickle=True)
    y_val = np.load(f"{base_path}/Benchmark Data/{data}/y_val.npy", allow_pickle=True)

    if((data != "analcatdata") and (data != "nursery")):
      default_N_train = np.load(f"{base_path}/Benchmark Data/{data}/N_train.npy", allow_pickle=True)
      C_train = np.load(f"{base_path}/{algo}/{data}/C_train.npy", allow_pickle=True)
      final_N_train = np.concatenate((default_N_train, C_train), axis=1)
      np.save(f"{path}/N_train.npy", final_N_train.astype(np.float32))

      default_N_val = np.load(f"{base_path}/Benchmark Data/{data}/N_val.npy", allow_pickle=True)
      C_val = np.load(f"{base_path}/{algo}/{data}/C_val.npy", allow_pickle=True)
      final_N_val = np.concatenate((default_N_val, C_val), axis=1)
      np.save(f"{path}/N_val.npy", final_N_val.astype(np.float32))

      default_N_test = np.load(f"{base_path}/Benchmark Data/{data}/N_test.npy", allow_pickle=True)
      C_test = np.load(f"{base_path}/{algo}/{data}/C_test.npy", allow_pickle=True)
      final_N_test = np.concatenate((default_N_test, C_test), axis=1)
      np.save(f"{path}/N_test.npy", final_N_test.astype(np.float32))

      info_dict = {
              "name": dir_name,
              "basename": dir_name,
              "split": 0,
              "task_type": "binclass" if y_train_df[0].nunique() == 2 else "multiclass",
              "n_classes": y_train_df[0].nunique(),
              "n_num_features": final_N_train.shape[1],
              "n_cat_features": 0,
              "train_size": final_N_train.shape[0],
              "val_size": final_N_val.shape[0],
              "test_size": final_N_test.shape[0]
          }

    else:
      C_train = np.load(f"{base_path}/{algo}/{data}/C_train.npy", allow_pickle=True)
      np.save(f"{path}/N_train.npy", C_train.astype(np.float32))

      C_val = np.load(f"{base_path}/{algo}/{data}/C_val.npy", allow_pickle=True)
      np.save(f"{path}/N_val.npy", C_val.astype(np.float32))

      C_test = np.load(f"{base_path}/{algo}/{data}/C_test.npy", allow_pickle=True)
      np.save(f"{path}/N_test.npy", C_test.astype(np.float32))

      info_dict = {
              "name": dir_name,
              "basename": dir_name,
              "split": 0,
              "task_type": "binclass" if y_train_df[0].nunique() == 2 else "multiclass",
              "n_classes": y_train_df[0].nunique(),
              "n_num_features": C_train.shape[1],
              "n_cat_features": 0,
              "train_size": C_train.shape[0],
              "val_size": C_val.shape[0],
              "test_size": C_test.shape[0]
          }

    np.save(f"{path}/y_train.npy", y_train.astype(np.int64))
    np.save(f"{path}/y_test.npy", y_test.astype(np.int64))
    np.save(f"{path}/y_val.npy", y_val.astype(np.int64))


    print(f"For {dir_name} =======")
    print(info_dict)
    print(f"shape of y_train = {y_train.shape}, y_val = {y_val.shape}, y_test = {y_test.shape} ")
    print()

    with open(f"{path}/info.json", 'w') as fp:
      json.dump(info_dict, fp)

In [ ]:
# Now create info.json files for default datasets and keep it in Benchmark Data folder

for data in tqdm.tqdm(data_list):

  y_train = np.load(f"{base_path}/Benchmark Data/{data}/y_train.npy", allow_pickle=True)
  y_train_df = pd.DataFrame(y_train)

  y_test = np.load(f"{base_path}/Benchmark Data/{data}/y_test.npy", allow_pickle=True)
  y_val = np.load(f"{base_path}/Benchmark Data/{data}/y_val.npy", allow_pickle=True)

  if((data != "analcatdata") and (data != "kr-vs-kp")):
    default_N_train = np.load(f"{base_path}/Benchmark Data/{data}/N_train.npy", allow_pickle=True)

    default_N_test = np.load(f"{base_path}/Benchmark Data/{data}/N_test.npy", allow_pickle=True)

    default_N_val = np.load(f"{base_path}/Benchmark Data/{data}/N_val.npy", allow_pickle=True)

    default_C_train = np.load(f"{base_path}/Benchmark Data/{data}/C_train.npy", allow_pickle=True)

    info_dict = {
                  "name": data,
                  "basename": data,
                  "split": 0,
                  "task_type": "binclass" if y_train_df[0].nunique() == 2 else "multiclass",
                  "n_classes": y_train_df[0].nunique(),
                  "n_num_features": default_N_train.shape[1],
                  "n_cat_features": default_C_train.shape[1],
                  "train_size": default_N_train.shape[0],
                  "val_size": default_N_val.shape[0],
                  "test_size": default_N_test.shape[0]
              }

  else:
    default_C_train = np.load(f"{base_path}/Benchmark Data/{data}/C_train.npy", allow_pickle=True)
    default_C_val = np.load(f"{base_path}/Benchmark Data/{data}/C_val.npy", allow_pickle=True)
    default_C_test = np.load(f"{base_path}/Benchmark Data/{data}/C_test.npy", allow_pickle=True)

    info_dict = {
                  "name": data,
                  "basename": data,
                  "split": 0,
                  "task_type": "binclass" if y_train_df[0].nunique() == 2 else "multiclass",
                  "n_classes": y_train_df[0].nunique(),
                  "n_num_features": 0,
                  "n_cat_features": default_C_train.shape[1],
                  "train_size": default_C_train.shape[0],
                  "val_size": default_C_val.shape[0],
                  "test_size": default_C_test.shape[0]
              }

  np.save(f"{path}/y_train.npy", y_train.astype(np.int64))
  np.save(f"{path}/y_test.npy", y_test.astype(np.int64))
  np.save(f"{path}/y_val.npy", y_val.astype(np.int64))


  print(f"For {data} =======")
  print(info_dict)
  print(f"shape of y_train = {y_train.shape}, y_val = {y_val.shape}, y_test = {y_test.shape} ")
  print()

  with open(f"{base_path}/Benchmark Data/{data}/info.json", 'w') as fp:
    json.dump(info_dict, fp)

In [ ]:
# Now copy default datasets intp Final Data Folder
for data in tqdm.tqdm(data_list):
    path = f"{base_path}/Final Data"
    os.makedirs(path, exist_ok=True)
    !cp -r '{base_path}/Benchmark Data/{data}' '{path}'

In [ ]:
# Type cast all categorical data to '<U26'

for data in tqdm.tqdm(data_list):
  default_C_train = np.load(f"{base_path}/Final Data/{data}/C_train.npy", allow_pickle=True)
  default_C_train = np.where(default_C_train == 'None', 'nan', default_C_train)

  default_C_test = np.load(f"{base_path}/Final Data/{data}/C_test.npy", allow_pickle=True)
  default_C_test = np.where(default_C_test == 'None', 'nan', default_C_test)

  default_C_val = np.load(f"{base_path}/Final Data/{data}/C_val.npy", allow_pickle=True)
  default_C_val = np.where(default_C_val == 'None', 'nan', default_C_val)

  np.save(f"{base_path}/Final Data/{data}/C_train.npy", default_C_train.astype('<U26'))
  np.save(f"{base_path}/Final Data/{data}/C_test.npy", default_C_test.astype('<U26'))
  np.save(f"{base_path}/Final Data/{data}/C_val.npy", default_C_val.astype('<U26'))


In [ ]:
for data in tqdm.tqdm(data_list):
  if((data != "analcatdata") and (data != "kr-vs-kp")):
    default_N_train = np.load(f"{base_path}/Final Data/{data}/N_train.npy", allow_pickle=True)

    default_N_test = np.load(f"{base_path}/Final Data/{data}/N_test.npy", allow_pickle=True)

    default_N_val = np.load(f"{base_path}/Final Data/{data}/N_val.npy", allow_pickle=True)

    np.save(f"{base_path}/Final Data/{data}/N_train.npy", default_N_train.astype(np.float32))
    np.save(f"{base_path}/Final Data/{data}/N_test.npy", default_N_test.astype(np.float32))
    np.save(f"{base_path}/Final Data/{data}/N_val.npy", default_N_val.astype(np.float32))

In [ ]:
!zip -r  "{base_path}/data.zip" "{base_path}/Final Data"